In [1]:
import pandas as pd

In [2]:
with open('./enz_flux_calculation.txt') as f:
    x = f.read().split('\n')
x = [i for i in x if i != '']

enzsyn_dict = dict(); enzload_dict = dict()
for i in x:
    enzid,v = i.split('\t')
    etype = enzid.split('-')[0]
    if etype == 'ENZSYN':
        if float(v) > 0:
            enzsyn_dict[enzid] = float(v)
    elif etype == 'ENZLOAD':
        if float(v) > 0:
            enzload_dict[enzid] = float(v)
    else:
        print('wtf')

In [3]:
#### Write enzyme synthesis constraints based on enzyme flux calculated from proteome
eqn_list = []; venzSyn = []

for k in enzsyn_dict.keys():
    # Constraints
    lhs = "v('" + k + "')"
    #rhs = "venzSyn('" + k + "') - enzfrac('" + k + "') * venzSlack"
    #rhs = "venzSyn('" + k + "') * (1 - venzDown + venzUp)"
    rhs = "venzSyn('" + k + "') * (1 - venzSlack)"
    eqn_list.append(lhs + ' =e= ' + rhs + ';')
    
    # Enzyme flux upper bounds as constants
    venzSyn.append("'" + k + "' " + str(enzsyn_dict[k]))
    
eqn_idx = ['enzSyn'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['enzSyn'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

venzSyn = ['/'] + venzSyn + ['/']

with open('./enz_calibration/enzSyn_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./enz_calibration/enzSyn_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))
with open('./enz_calibration/enzSyn_venzSyn.txt', 'w') as f:
    f.write('\n'.join(venzSyn))

In [5]:
#### Write constraints with slack variables applied to rxns,
# related to rxns associated with enzymes not presence in dataset
df_eqn = pd.read_excel('../scRBA_build_GAMS_model/model/RBA_stoichiometry.xlsx')
df_eqn.index = df_eqn.id.to_list()
idx = df_eqn[df_eqn.id.str.contains('^ENZLOAD-')].index

eqn_list = []; noEnzList = []

for i in idx:
    if i not in enzload_dict.keys():
        rxnid = 'RXN-' + i.split('-', maxsplit=1)[1]
        
        # Record flux variables that are in consideration
        noEnzList.append("'" + rxnid + "'")
        
        # Constraints
        lhs = "v('" + rxnid + "')"
        rhs = "dv('" + rxnid + "')"
        eqn_list.append(lhs + ' =l= ' + rhs + ';')
        
eqn_idx = ['noEnz'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['noEnz'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

noEnzList = ['/'] + noEnzList + ['/']

with open('./enz_calibration/noEnz_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./enz_calibration/noEnz_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))
with open('./enz_calibration/noEnz_dvset.txt', 'w') as f:
    f.write('\n'.join(noEnzList))

In [10]:
df_enz = pd.read_excel('../scRBA_build_GAMS_model/input/ENZYME_stoich_curation_2021-10-01.xlsx')
df_enz.index = df_enz.id.to_list()

In [11]:
df_enz

,id,rxn_src,enz,gpr,protein_stoich,subunit_comments,status,MW (g/mmol)
RXN-BTDDH_c_FWD-YAL060W,RXN-BTDDH_c_FWD-YAL060W,BTDDH_c,YAL060W,YAL060W,YAL060W:2,NaN,protStoichAsgnAuto,91.17410
RXN-BTDDH_c_REV-YAL060W,RXN-BTDDH_c_REV-YAL060W,BTDDH_c,YAL060W,YAL060W,YAL060W:2,NaN,protStoichAsgnAuto,91.17410
RXN-13GS_c_FWD-GSC2RHO1,RXN-13GS_c_FWD-GSC2RHO1,13GS_c,GSC2RHO1,YLR342W and YPR165W,"YLR342W:1,YPR165W:1","Component of the 1,3-beta-glucan synthase (GS)...",protStoichAsgnManual,238.33867
RXN-13GS_c_FWD-GSC1RHO1,RXN-13GS_c_FWD-GSC1RHO1,13GS_c,GSC1RHO1,YGR032W and YPR165W,"YGR032W:1,YPR165W:1","Component of the 1,3-beta-glucan synthase (GS)...",protStoichAsgnManual,241.31742
RXN-16GS_c_FWD-YPR159W,RXN-16GS_c_FWD-YPR159W,16GS_c,YPR159W,YPR159W,YPR159W:1,The cytoplasmic domain interacts with the acti...,protStoichAsgnManual,81.57498
...,...,...,...,...,...,...,...,...
RXN-AKGDH_m_FWD-KGDCPLX,RXN-AKGDH_m_FWD-KGDCPLX,AKGDH_m,KGDCPLX,YDR148C and YFL018C and YIL125W and YFR049W,"YDR148C:1,YFL018C:1,YIL125W:1,YFR049W:1",LPD1 is a homodimer. Eukaryotic pyruvate dehyd...,protStoichAsgnManual,243.20661
RXN-ETFOXRq6_m_FWD-CIR1AIM45,RXN-ETFOXRq6_m_FWD-CIR1AIM45,ETFOXRq6_m,CIR1AIM45,YGR207C and YPR004C,"YGR207C:1,YPR004C:1",Heterodimer of an alpha and a beta subunit (By...,protStoichAsgnManual,69.86101
RXN-PYDX5PS_c_FWD-SNZ1SNO1,RXN-PYDX5PS_c_FWD-SNZ1SNO1,PYDX5PS_c,SNZ1SNO1,YMR096W and YMR095C,"YMR096W:1,YMR095C:1",Homohexamer (PubMed:19523954). Interacts with ...,protStoichAsgnManual,60.64526
RXN-PYDX5PS_c_FWD-SNZ2SNO1,RXN-PYDX5PS_c_FWD-SNZ2SNO1,PYDX5PS_c,SNZ2SNO1,YNL333W and YMR095C,"YNL333W:1,YMR095C:1",Homohexamer (By similarity). Interacts with TH...,protStoichAsgnManual,60.30611


In [8]:
enzsyn_dict.values()

dict_values([1.3631584946e-07, 7.8363791598e-07, 1.1655739959999999e-08, 1.7463449028e-07, 6.840051462000001e-08, 5.0859795418e-07, 1.598324954e-08, 4.102389734e-08, 1.6890250619e-07, 9.4793453551e-07, 3.034967321e-07, 4.38910558132e-06, 3.335668548e-08, 1.70826145373e-06, 3.0533719639999997e-08, 5.146804812e-08, 3.3235411444000003e-07, 2.264281336e-08, 4.5545950216999996e-07, 6.340415432e-08, 3.027755551e-08, 1.4653904568e-07, 4.977605898e-07, 1.3270754302000002e-07, 3.5680529044999996e-07, 3.77053961203e-06, 1.681008313e-08, 1.681008313e-08, 2.0754970500000002e-08, 1.0193984019000001e-07, 1.0598760105e-07, 2.1701593379999998e-08, 2.94777377429e-06, 3.13657371e-09, 3.0046380885600003e-06, 4.40990472e-09, 3.352803655e-08, 1.59260319e-08, 4.849996971e-08, 2.5411714011e-06, 2.697019245e-08, 9.925422708999999e-08, 1.5693532329000002e-07, 6.28675014e-09, 2.052308998e-08, 1.30913500434e-06, 1.9481730483000002e-07, 9.5769115039e-07, 1.0767775677e-07, 3.721889978183e-05, 3.478684597e-08, 1.86